In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np
from sklearn.utils import class_weight

# تنظیم DataGenerator برای پیش‌پردازش و افزایش داده‌ها
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

# بارگذاری داده‌ها از مسیر دیتاست
train_generator = train_datagen.flow_from_directory(
    'processed_images/',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'processed_images/',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# محاسبه وزن‌های کلاس‌ها برای مدیریت نابرابری
class_weights = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = dict(enumerate(class_weights))

# بارگذاری مدل پیش‌آموزش‌دیده ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# اضافه کردن لایه‌های سفارشی به مدل
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))  # 4 کلاس: Tumor, Stone, Cyst, Normal

# فریز کردن لایه‌های مدل پایه
for layer in base_model.layers:
    layer.trainable = False

# کامپایل مدل
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# آموزش مدل
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 32,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // 32,
    class_weight=class_weights
)

# ارزیابی مدل
loss, accuracy = model.evaluate(validation_generator)
print(f'دقت مدل: {accuracy}')

Found 9959 images belonging to 4 classes.
Found 2487 images belonging to 4 classes.
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 47s 0us/step


c:\Users\Ayandegan\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
311/311 ━━━━━━━━━━━━━━━━━━━━ 1083s 3s/step - accuracy: 0.4007 - loss: 3.9001 - val_accuracy: 0.3145 - val_loss: 1.3962
Epoch 2/20
  1/311 ━━━━━━━━━━━━━━━━━━━━ 13:55 3s/step - accuracy: 0.4062 - loss: 1.2771

c:\Users\Ayandegan\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


311/311 ━━━━━━━━━━━━━━━━━━━━ 201s 641ms/step - accuracy: 0.4062 - loss: 1.2771 - val_accuracy: 0.3243 - val_loss: 1.3913
Epoch 3/20
311/311 ━━━━━━━━━━━━━━━━━━━━ 1075s 3s/step - accuracy: 0.4178 - loss: 1.3182 - val_accuracy: 0.5593 - val_loss: 1.2586
Epoch 4/20
311/311 ━━━━━━━━━━━━━━━━━━━━ 199s 633ms/step - accuracy: 0.5312 - loss: 1.1764 - val_accuracy: 0.5097 - val_loss: 1.2963
Epoch 5/20
311/311 ━━━━━━━━━━━━━━━━━━━━ 1059s 3s/step - accuracy: 0.3557 - loss: 1.2976 - val_accuracy: 0.5341 - val_loss: 1.1889
Epoch 6/20
311/311 ━━━━━━━━━━━━━━━━━━━━ 199s 626ms/step - accuracy: 0.3438 - loss: 1.3193 - val_accuracy: 0.5657 - val_loss: 1.1632
Epoch 7/20
311/311 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.3528 - loss: 1.2708